In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import imutils
import easyocr

def anpr(upload_img):
    #step1
    # matplotlib interprets images in RGB format, but OpenCV uses BGR format
    # so to convert the image so that it's properly loaded, convert it before loading
    img = cv2.imread(f"Upload/{upload_img}")
    print(f'anpr.upload img:{upload_img}')
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # plt.imshow(cv2.cvtColor(gray, cv2.COLOR_BGR2RGB))

    #step2
    bfilter = cv2.bilateralFilter(gray, 11, 17, 17) #Noise reduction
    edged = cv2.Canny(bfilter, 30, 200) #Edge detection
    # plt.imshow(cv2.cvtColor(edged, cv2.COLOR_BGR2RGB))

    #step3
    keypoints = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(keypoints)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]

        #location
    location = None
    for contour in contours:
        approx = cv2.approxPolyDP(contour, 10, True)
        if len(approx) == 4:
             location = approx
             break
    # print(location)
    mask = np.zeros(gray.shape, np.uint8)
    new_image = cv2.drawContours(mask, [location], 0,255, -1)
    new_image = cv2.bitwise_and(img, img, mask=mask)
    # plt.imshow(cv2.cvtColor(new_image, cv2.COLOR_BGR2RGB))

        #crop_image
    (x,y) = np.where(mask==255)
    (x1, y1) = (np.min(x), np.min(y))
    (x2, y2) = (np.max(x), np.max(y))
    cropped_image = gray[x1:x2+1, y1:y2+1]
    # plt.imshow(cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB))

    #step4
    reader = easyocr.Reader(['en'])
    result = reader.readtext(cropped_image)

    #step5
    text = result[0][-2]
    font = cv2.FONT_HERSHEY_SIMPLEX
    res = cv2.putText(img, text=text, org=(approx[0][0][0], approx[1][0][1]+60), fontFace=font, fontScale=1, color=(0,255,0), thickness=2, lineType=cv2.LINE_AA)
    res = cv2.rectangle(img, tuple(approx[0][0]), tuple(approx[2][0]), (0,255,0),3)
    plt.imshow(cv2.cvtColor(res, cv2.COLOR_BGR2RGB))
    plt.savefig(f"Results/{upload_img}")
    print(f'anpr.save:{ plt.savefig(f"Results/{upload_img}")}')
    return text


import os
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r"D:\tesseract\tesseract.exe"
from flask import Flask, render_template, redirect,url_for, request, redirect, jsonify, Response
import requests
app = Flask(__name__)
app.config["IMAGE_UPLOADS"] = "D:/github/ANPRwithPython/Upload/"
app.config["IMAGE_RESULTS"] = "D:/github/ANPRwithPython/Results/"


@app.route("/")
@app.route("/home")
def home():
    return render_template("home.html")

@app.route("/test")
def test():
    return render_template("test.html")

# Pokemons =["Pikachu", "Charizard", "Squirtle", "Jigglypuff",
#            "Bulbasaur", "Gengar", "Charmander", "Mew", "Lugia", "Gyarados"]
# @app.route("/result")
# def result():
#     return render_template("result.html",len = len(Pokemons), Pokemons = Pokemons)

@app.route('/upload-image', methods=['GET', 'POST'])
def upload_image():
    if request.method == "POST":
        if request.files:
            image = request.files["img"]
            image.save(os.path.join(app.config["IMAGE_UPLOADS"], image.filename))
            print(f"Route.ImageName:{image.filename}")
            text = anpr(image.filename)

            # Open a file with access mode 'a'
            file_object = open(f'texts/text.txt', 'a')
            # Append 'hello' at the end of file
            file_object.write(f'results/{image.filename}:{text}\n')

            # Close the file
            file_object.close()

            return render_template("test.html", uploaded_image=image.filename, results_image=image.filename,result= text)
    return render_template("test.html")


@app.route('/uploads/<filename>')
def send_uploaded_file(filename=''):
    from flask import send_from_directory
    return send_from_directory(app.config["IMAGE_UPLOADS"], filename)

@app.route('/results/<filename>')
def send_result_file(filename=''):
    from flask import send_from_directory
    return send_from_directory(app.config["IMAGE_RESULTS"], filename)

@app.route('/result')
def detection():

    with open('texts/text.txt') as event_log:
        event_lines = event_log.readlines()

        results = []
        events = len(event_lines)

        for x in range(events):
            results.append(event_lines[x].split(":"))
        #print()


    basepath = f"results"
    dir = os.walk(basepath)
    file_list = []

    for path, subdirs, files in dir:
        for file in files:
            temp = os.path.join(path + '/', file)
            print(temp)
            file_list.append(temp)
            print(file_list)
    return render_template('result.html', len = len(file_list), hists=file_list,events=events, results=results)


if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Mar/2022 06:16:26] "GET /result HTTP/1.1" 200 -


results/Cars0.png
['results/Cars0.png']
results/Cars1.png
['results/Cars0.png', 'results/Cars1.png']
results/Cars11.png
['results/Cars0.png', 'results/Cars1.png', 'results/Cars11.png']
results/Cars3.png
['results/Cars0.png', 'results/Cars1.png', 'results/Cars11.png', 'results/Cars3.png']
results/Cars4.png
['results/Cars0.png', 'results/Cars1.png', 'results/Cars11.png', 'results/Cars3.png', 'results/Cars4.png']
results/Cars6.png
['results/Cars0.png', 'results/Cars1.png', 'results/Cars11.png', 'results/Cars3.png', 'results/Cars4.png', 'results/Cars6.png']


127.0.0.1 - - [29/Mar/2022 06:16:31] "GET /test HTTP/1.1" 200 -
127.0.0.1 - - [29/Mar/2022 06:16:31] "GET /uploads/ HTTP/1.1" 404 -
127.0.0.1 - - [29/Mar/2022 06:16:31] "GET /results/ HTTP/1.1" 404 -
127.0.0.1 - - [29/Mar/2022 06:16:32] "GET /result HTTP/1.1" 200 -


results/Cars0.png
['results/Cars0.png']
results/Cars1.png
['results/Cars0.png', 'results/Cars1.png']
results/Cars11.png
['results/Cars0.png', 'results/Cars1.png', 'results/Cars11.png']
results/Cars3.png
['results/Cars0.png', 'results/Cars1.png', 'results/Cars11.png', 'results/Cars3.png']
results/Cars4.png
['results/Cars0.png', 'results/Cars1.png', 'results/Cars11.png', 'results/Cars3.png', 'results/Cars4.png']
results/Cars6.png
['results/Cars0.png', 'results/Cars1.png', 'results/Cars11.png', 'results/Cars3.png', 'results/Cars4.png', 'results/Cars6.png']


...

results/Cars0.png
['results/Cars0.png']
results/Cars1.png
['results/Cars0.png', 'results/Cars1.png']
results/Cars11.png
['results/Cars0.png', 'results/Cars1.png', 'results/Cars11.png']
results/Cars3.png
['results/Cars0.png', 'results/Cars1.png', 'results/Cars11.png', 'results/Cars3.png']
results/Cars4.png
['results/Cars0.png', 'results/Cars1.png', 'results/Cars11.png', 'results/Cars3.png', 'results/Cars4.png']
results/Cars6.png
['results/Cars0.png', 'results/Cars1.png', 'results/Cars11.png', 'results/Cars3.png', 'results/Cars4.png', 'results/Cars6.png']


127.0.0.1 - - [29/Mar/2022 06:17:24] "GET /test HTTP/1.1" 200 -
127.0.0.1 - - [29/Mar/2022 06:17:24] "GET /uploads/ HTTP/1.1" 404 -
127.0.0.1 - - [29/Mar/2022 06:17:24] "GET /results/ HTTP/1.1" 404 -
127.0.0.1 - - [29/Mar/2022 06:17:26] "GET /result HTTP/1.1" 200 -


results/Cars0.png
['results/Cars0.png']
results/Cars1.png
['results/Cars0.png', 'results/Cars1.png']
results/Cars11.png
['results/Cars0.png', 'results/Cars1.png', 'results/Cars11.png']
results/Cars3.png
['results/Cars0.png', 'results/Cars1.png', 'results/Cars11.png', 'results/Cars3.png']
results/Cars4.png
['results/Cars0.png', 'results/Cars1.png', 'results/Cars11.png', 'results/Cars3.png', 'results/Cars4.png']
results/Cars6.png
['results/Cars0.png', 'results/Cars1.png', 'results/Cars11.png', 'results/Cars3.png', 'results/Cars4.png', 'results/Cars6.png']


127.0.0.1 - - [29/Mar/2022 06:17:42] "GET /home HTTP/1.1" 200 -
127.0.0.1 - - [29/Mar/2022 06:17:45] "GET /test HTTP/1.1" 200 -
127.0.0.1 - - [29/Mar/2022 06:17:45] "GET /uploads/ HTTP/1.1" 404 -
127.0.0.1 - - [29/Mar/2022 06:17:45] "GET /results/ HTTP/1.1" 404 -
127.0.0.1 - - [29/Mar/2022 06:17:46] "GET /result HTTP/1.1" 200 -


results/Cars0.png
['results/Cars0.png']
results/Cars1.png
['results/Cars0.png', 'results/Cars1.png']
results/Cars11.png
['results/Cars0.png', 'results/Cars1.png', 'results/Cars11.png']
results/Cars3.png
['results/Cars0.png', 'results/Cars1.png', 'results/Cars11.png', 'results/Cars3.png']
results/Cars4.png
['results/Cars0.png', 'results/Cars1.png', 'results/Cars11.png', 'results/Cars3.png', 'results/Cars4.png']
results/Cars6.png
['results/Cars0.png', 'results/Cars1.png', 'results/Cars11.png', 'results/Cars3.png', 'results/Cars4.png', 'results/Cars6.png']
